### Семинар 3
В данном семинаре будут рассмотрены следующие темы:
 - Постановка задачи регрессии
 - Метрики задачи регрессии
 - Постановка задачи классификации
 - Метрики задачи классификации
 - Постановка задачи кластеризации
 - Объяснение алгоритма KNN
 - Имплементация алгоритма KNN
 - Оценка качества реализованного алгоритма


### Постановка задачи регрессии
Задачи регрессии — это задачи машинного обучения при использовании числового целевого столбца. 

### Метрики задачи регрессии
- MAE
- MSE
- RMSE
- MAPE
- R2

In [ ]:
import numpy as np
from sklearn import metrics

In [ ]:
y = np.array([100, 150, 115, 125])
y_pred = np.array([96, 143, 120, 120])

In [ ]:
# MAE
metrics.mean_absolute_error(y, y_pred)

In [ ]:
np.abs(y - y_pred).sum() / len(y)

In [ ]:
# MSE
metrics.mean_squared_error(y, y_pred)

In [ ]:
((y - y_pred) ** 2).sum() / len(y)

In [ ]:
# RMSE
metrics.root_mean_squared_error(y, y_pred)

In [ ]:
np.sqrt(((y - y_pred) ** 2).sum() / len(y))

In [ ]:
# R2
metrics.r2_score(y, y_pred)

In [ ]:
1 - (((y_pred - y) ** 2).sum() / ((y - y.mean()) ** 2).sum())

### Постановка задачи Классификации
Классификация - задача, в которой множество объектов необходимо разделить некоторым образом на классы, при этом задано конечное множество объектов, для которых известно, к каким классам они относятся (выборка), но классовая принадлежность остальных объектов неизвестна.

### Метрики задачи классификации

- Accuracy
- Precision
- Recall
- F1-score
- ROC-AUC

In [ ]:
y =      np.array([1, 0, 1, 1, 1, 0, 0])
y_pred = np.array([0, 0, 1, 1, 1, 1, 1])

In [ ]:
# Accuracy
metrics.accuracy_score(y, y_pred)

In [ ]:
(y == y_pred).mean()

In [15]:
# Precision
metrics.precision_score(y, y_pred)

np.float64(0.6)

In [16]:
tp = ((y == 1) & (y_pred == 1)).sum()
fp = ((y == 0) & (y_pred == 1)).sum()
fn = y.sum()

In [17]:
precision = tp / (tp + fp)
precision

np.float64(0.6)

In [18]:
# Recall
metrics.recall_score(y, y_pred)

np.float64(0.75)

In [19]:
recall = tp / fn
recall

np.float64(0.75)

In [20]:
# F1
metrics.f1_score(y, y_pred)

np.float64(0.6666666666666666)

In [21]:
f1 = 2 * (precision * recall) / (precision + recall)
f1

np.float64(0.6666666666666665)

In [22]:
y = np.array([1, 0, 1, 0])
y_pred = np.array([0, 0, 1, 0])

In [23]:
# ROC-AUC
metrics.roc_auc_score(y, y_pred)

np.float64(0.75)

In [ ]:
print(y_pred, y, sep="\n")

Упорядочим наблюдения по **убыванию** ответов алгоритма.

In [ ]:
y = np.array([1, 1, 0, 0])
y_pred = np.array([1, 0, 0, 0])
print(y, y_pred, sep="\n")

Разобьём единичный квадрат на $(m, n)$ частей, где $m$ – число 1 в $y$, $n$ – число нулей. Стартуем из точки $(0, 0)$. Если значение $y$ равно 1, делаем шаг вверх, а если 0 – вправо. Понятно, что конечная точка нашего маршрута – точка $(1, 1)$.

**Важный момент:** если у нескольких объектов значения предсказаний равны, а $y$ – различны, то мы должны сделать ход \"по диагонали\".

Если построить кривую по этому алгоритму, то получим:

In [ ]:
y = np.array([1, 0, 0, 0])
y_pred = np.array([0, 1, 0, 0])
print(y, y_pred, sep="\n")

### Алгоритм KNN

![image info](https://raw.githubusercontent.com/hse-ds/iad-intro-ds/7062d0dba527f9c24e7c54d35bca256907ac722d/2023/seminars/sem05_sklearn_knn/static/k_grid.png)

In [24]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()

In [25]:
X, y = data['data'], data['target']

In [27]:
X.shape

(569, 30)

In [29]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

### Деление на обучающую и тестовую выборки

In [105]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=True)

In [107]:
from sklearn.neighbors import KNeighborsClassifier

In [108]:
model = KNeighborsClassifier(n_neighbors=3)

In [109]:
model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [110]:
preds = model.predict(X_test)

In [111]:
preds

array([0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0])

In [112]:
metrics.f1_score(y_test, preds)

np.float64(0.9378531073446328)

In [122]:
from sklearn.preprocessing import StandardScaler

In [123]:
class KNN:
    def __init__(self, k) -> None:
        self.k = k
        self.scaler = StandardScaler()

    def fit(self, X_train, y_train):
        self.scaler.fit(X_train)
        self.X_train = self.scaler.transform(X_train)
        self.y_train = y_train

    def euclidian_distance(self, a, b):
        return np.sqrt(((a - b)**2).sum())
    
    def predict(self, X_test):

        transformed_X_test = self.scaler.transform(X_test)

        distance_matrix = np.zeros((X_test.shape[0], X_train.shape[0]))
        for i in range(len(X_test)):
            for j in range(len(self.X_train)):
                distance_matrix[i][j] = self.euclidian_distance(transformed_X_test[i], self.X_train[j])
        
        idx_matrix = np.argsort(distance_matrix, axis=1)[:, :self.k]
        y_pred_initial = self.y_train[idx_matrix]
        y_pred = (y_pred_initial.mean(axis=1)>=0.5).astype(int)

        return y_pred


In [124]:
knn = KNN(3)

In [125]:
knn.fit(X_train, y_train)

In [126]:
knn.predict(X_test)

array([0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0])

In [127]:
metrics.f1_score(y_test, knn.predict(X_test))

np.float64(0.9772727272727273)

In [119]:
import pandas as pd

In [121]:
pd.DataFrame(X_train).mean()

0      14.070838
1      19.353920
2      91.562512
3     647.789906
4       0.096013
5       0.103468
6       0.087411
7       0.047811
8       0.180931
9       0.062643
10      0.401187
11      1.221517
12      2.829413
13     40.133911
14      0.006927
15      0.025314
16      0.031897
17      0.011546
18      0.020819
19      0.003784
20     16.201251
21     25.763615
22    106.707230
23    872.481690
24      0.131585
25      0.253592
26      0.272022
27      0.112971
28      0.291034
29      0.083708
dtype: float64

In [58]:
X_train.shape, X_test.shape

((426, 30), (143, 30))